# Probabilistic Modeling with different Keras Backends (tf_keras)

In [33]:
import numpy as np
import matplotlib.pyplot as plt

# Generate training data
np.random.seed(42)
x_train = np.random.uniform(-10, 10, (1000, 1)).astype(np.float32)
y_train = (2 * x_train + np.random.normal(0, np.abs(x_train) + 1, (1000, 1))).astype(np.float32)

# Using tf_keras (Nov, 2024)

Before Keras 3.0, Keras was part of tensorflow and had a tight integration. Especially the output of a Keras model `model(x)` could be a probability distribution (tfp_distribution). From that you could: sample, calculate the pdf, cdf and so on. Downsides you have to install tfp, and that causes quite some pain in the past. 

```python
dist = model(x_test)
dist.sample(5) # Sample from the distribution
dist.cdf(0.0) # # Compute CDF values (100, 1)
...
```

The trick is the `tfp.layers.DistributionLambda` layer, which integrates a distrubution in the keras layer. It's really beautiful to calculate the NLL see code below.

In [39]:
import tensorflow_probability as tfp
import numpy as np
import tf_keras #<------ Note that you are not using tf.keras
import tensorflow as tf

print(F'tf_keras version {tf_keras.__version__}')
print(F'tfp version      {tfp.__version__}')
print(F'tf version       {tf.__version__}')

tfd = tfp.distributions
tfpl = tfp.layers

def my_distribution(inputs):
    return tfp.distributions.Normal(loc=inputs[..., :1],
                                    scale=1e-3 + tf.math.softplus(0.05 * inputs[...,1:]))

inputs = tf_keras.Input(shape=(1,))
hidden = tf_keras.layers.Dense(10, activation='relu')(inputs)
outputs = tf_keras.layers.Dense(2)(hidden)
out_dist = tfp.layers.DistributionLambda(my_distribution)(outputs)
model = tf_keras.Model(inputs=inputs, outputs=out_dist)

# Look how nicely the likelihood can be calculated
def nll(y_true, cpd):
    return -cpd.log_prob(y_true)

model.compile(optimizer=tf_keras.optimizers.Adam(learning_rate=0.01), loss=nll)

# Train the model
model.fit(x_train, y_train, epochs=2, verbose=False)

# Predict and get the distribution
x_test = np.linspace(-10, 10, 100).reshape(-1, 1)
dist = model(x_test)

print(dist.sample(5).shape) # Sample from the distribution
print(dist.cdf(0.0).shape) # # Compute CDF values (100, 1)

tf_keras version 2.17.0
tfp version      0.24.0
tf version       2.17.1
(5, 100, 1)
(100, 1)
